<a href="https://colab.research.google.com/github/mawhy/OpenCV/blob/master/Image_Registration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Processing CookBook
## Image Registration
In Polish means  "nakładanie obrazów"

In [0]:
!git clone https://github.com/PacktPublishing/Python-Image-Processing-Cookbook.git
%cp -av "/content/Python-Image-Processing-Cookbook/Chapter 05/images/" "/content/"
%cp -av "/content/Python-Image-Processing-Cookbook/Chapter 05/models/" "/content/"
%rm -rf "/content/Python-Image-Processing-Cookbook"

### Medical Image Registration with SimpleITK

In [0]:
!pip install SimpleITK

In [0]:
!pip install 
!pip install opencv-python==4.2.0.34
!pip install opencv-contrib-python==4.2.0.34

In [0]:
# https://stackoverflow.com/questions/41692063/what-is-the-difference-between-image-registration-and-image-alignment
# https://www.insight-journal.org/rire/download_training_data.php
# https://itk.org/Wiki/SimpleITK/Tutorials/MICCAI2015
%matplotlib inline
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

fixed_image =  sitk.ReadImage("images/ct_scan_11.jpg", sitk.sitkFloat32)
moving_image = sitk.ReadImage("images/mr_T1_06.jpg", sitk.sitkFloat32) 

fixed_image_array = sitk.GetArrayFromImage(fixed_image)
moving_image_array = sitk.GetArrayFromImage(moving_image)
print(fixed_image_array.shape, moving_image_array.shape)
plt.figure(figsize=(20,10))
plt.gray()
plt.subplot(131), plt.imshow(fixed_image_array), plt.axis('off'), plt.title('CT-Scan Image', size=20)
plt.subplot(132), plt.imshow(moving_image_array), plt.axis('off'), plt.title('MRI-T1 Image', size=20)
plt.subplot(133), plt.imshow(0.6*fixed_image_array + 0.4*moving_image_array), plt.axis('off'), plt.title('Initial Alignment', size=20)
plt.show()

In [0]:
np.random.seed(2)
registration_method = sitk.ImageRegistrationMethod()

initial_transform = sitk.CenteredTransformInitializer(fixed_image, moving_image, sitk.Similarity2DTransform())
   
# Similarity metric settings.
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

# Optimizer settings.
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
registration_method.SetOptimizerScalesFromPhysicalShift()

# Setup for the multi-resolution framework.            
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# Don't optimize in-place, we would possibly like to run this cell multiple times.
registration_method.SetInitialTransform(initial_transform, inPlace=False)

final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                               sitk.Cast(moving_image, sitk.sitkFloat32))

In [0]:
#print(final_transform)
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed_image);
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(100)
resampler.SetTransform(final_transform)
 
out = resampler.Execute(moving_image)
simg1 = sitk.Cast(sitk.RescaleIntensity(fixed_image), sitk.sitkUInt8)
simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
cimg = sitk.Compose(simg1, simg2, simg1//2.+simg2//2.)

plt.figure(figsize=(20,10))
plt.gray()
plt.subplot(131), plt.imshow(fixed_image_array), plt.axis('off'), plt.title('CT-Scan Image', size=20)
plt.subplot(132), plt.imshow(sitk.GetArrayFromImage(out)), plt.axis('off'), plt.title('Transformed MRI-T1 Image', size=20)
plt.subplot(133), plt.imshow(sitk.GetArrayFromImage(cimg)), plt.axis('off'), plt.title('Final Alignment', size=20)
plt.show()

In [0]:
# https://www.insight-journal.org/rire/download_training_data.php
# https://itk.org/Wiki/SimpleITK/Tutorials/MICCAI2015
import SimpleITK as sitk
import matplotlib.pyplot as plt

fixed =  sitk.ReadImage("images/mr_T1_01.jpg", sitk.sitkFloat32)
moving = sitk.ReadImage("images/mr_T1_01_trans.jpg", sitk.sitkFloat32) 

R = sitk.ImageRegistrationMethod()
R.SetMetricAsMeanSquares()
R.SetOptimizerAsRegularStepGradientDescent(4.0, .01, 200 )
R.SetInterpolator(sitk.sitkLinear)
transfo = sitk.CenteredTransformInitializer(fixed, moving, sitk.Euler2DTransform())
R.SetInitialTransform(transfo)
outTx1 = R.Execute(fixed, moving)
print(outTx1)
print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
print("Number of iterations: {0}".format(R.GetOptimizerIteration()))
R.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
R.SetOptimizerAsRegularStepGradientDescent(4.0, .01, 200 )
R.SetInitialTransform(transfo)
outTx2 = R.Execute(fixed, moving)
print(outTx2)
print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
print("Number of iterations: {0}".format(R.GetOptimizerIteration()))

#sitk.WriteTransform(outTx, 'transfo_final.tfm')
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(100)
resampler.SetTransform(outTx1)
out1 = resampler.Execute(moving)
moving_image_array_trans1 = sitk.GetArrayFromImage(out1)
simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
simg2 = sitk.Cast(sitk.RescaleIntensity(out1), sitk.sitkUInt8)
cimg1_array = sitk.GetArrayFromImage(sitk.Compose(simg1, simg2, simg1//2.+simg2//2.))
resampler.SetTransform(outTx2)
out2 = resampler.Execute(moving)
moving_image_array_trans2 = sitk.GetArrayFromImage(out2)
simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
simg2 = sitk.Cast(sitk.RescaleIntensity(out2), sitk.sitkUInt8)
cimg2_array = sitk.GetArrayFromImage(sitk.Compose(simg1, simg2, simg1//2.+simg2//2.))
       
fixed_image_array = sitk.GetArrayFromImage(fixed)
moving_image_array = sitk.GetArrayFromImage(moving)
print(fixed_image_array.shape, moving_image_array.shape)
plt.figure(figsize=(20,30))
plt.gray()
plt.subplots_adjust(0,0,1,1,0.075,0.01)
plt.subplot(321), plt.imshow(fixed_image_array), plt.axis('off'), plt.title('MR-T1 Image', size=20)
plt.subplot(322), plt.imshow(moving_image_array), plt.axis('off'), plt.title('Shifted MR_T1 Image', size=20)
plt.subplot(323), plt.imshow(fixed_image_array - moving_image_array_trans1), plt.axis('off'), plt.title('Difference Images (MeanSquare)', size=20)
plt.subplot(324), plt.imshow(fixed_image_array - moving_image_array_trans2), plt.axis('off'), plt.title('Difference Images (MutualInformation)', size=20)
plt.subplot(325), plt.imshow(cimg1_array), plt.axis('off'), plt.title('Aligned Images (MeanSquare)', size=20)
plt.subplot(326), plt.imshow(cimg2_array), plt.axis('off'), plt.title('Aligned Images (MutualInformation)', size=20)
plt.show()

In [0]:
checkerboard = sitk.CheckerBoardImageFilter()
before_reg_image = checkerboard.Execute (fixed, moving)
after_reg_image = checkerboard.Execute (fixed, out2)
plt.figure(figsize=(20,10))
plt.gray()
plt.subplot(121), plt.imshow(sitk.GetArrayFromImage(before_reg_image)), plt.axis('off'), plt.title('Checkerboard before Registration Image', size=20)
plt.subplot(122), plt.imshow(sitk.GetArrayFromImage(after_reg_image)), plt.axis('off'), plt.title('Checkerboard After Registration Image', size=20)
plt.show()

### Image Alignment with ECC algorithm
[Good articles](https://www.learnopencv.com/image-alignment-ecc-in-opencv-c-python/)

In [0]:
import cv2
print(cv2.__version__)
# 4.2.0
import numpy as np
import matplotlib.pylab as plt
 
def compute_gradient(im) :
    grad_x = cv2.Sobel(im,cv2.CV_32F,1,0,ksize=3)
    grad_y = cv2.Sobel(im,cv2.CV_32F,0,1,ksize=3)
    grad = cv2.addWeighted(np.absolute(grad_x), 0.5, np.absolute(grad_y), 0.5, 0)
    return grad

im_unaligned =  cv2.imread("images/me_unaligned.png")

height, width = im_unaligned.shape[:2]
print(height, width)

channels = ['B', 'G', 'R']

plt.figure(figsize=(30,12))
plt.gray()
plt.subplot(1,4,1), plt.imshow(cv2.cvtColor(im_unaligned, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Unaligned Image', size=20)
for i in range(3):
    plt.subplot(1,4,i+2), plt.imshow(im_unaligned[...,i]), plt.axis('off'), plt.title(channels[i], size=20)
plt.suptitle('Unaligned Image and Color Channels', size=30)
plt.show()

# Initialize the output image with a copy of the input image
im_aligned = im_unaligned.copy() 

# Define motion model
warp_mode = cv2.MOTION_HOMOGRAPHY

# Set the warp matrix to identity.
warp_matrix = np.eye(3, 3, dtype=np.float32) if warp_mode == cv2.MOTION_HOMOGRAPHY else np.eye(2, 3, dtype=np.float32)

# Set the stopping criteria for the algorithm.
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 500,  1e-6)

# The blue and green channels will be aligned to the red channel, so compute the gradient of the red channel first
im_grad2 = compute_gradient(im_unaligned[...,2])

# Warp the blue and green channels to the red channel
for i in range(2) :
    print('Processing Channel {}...'.format(channels[i]))
    (cc, warp_matrix) = cv2.findTransformECC (im_grad2, compute_gradient(im_unaligned[...,i]),warp_matrix, warp_mode, criteria, None, 5)

    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        # Perspective warp - transformation is a Homography
        im_aligned[...,i] = cv2.warpPerspective (im_unaligned[...,i], warp_matrix, (width,height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    else :
        # Affine warp - transformation is not a Homography
        im_aligned[...,i] = cv2.warpAffine(im_unaligned[...,i], warp_matrix, (width, height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);
    print (warp_matrix)
    
channels = ['B', 'G', 'R']
plt.figure(figsize=(30,12))
plt.subplot(1,4,1), plt.imshow(cv2.cvtColor(im_aligned, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Aligned Image (ECC)', size=20)
for i in range(3):
    plt.subplot(1,4,i+2), plt.imshow(im_aligned[...,i]), plt.axis('off'), plt.title(channels[i], size=20)
plt.suptitle('Aligned Image and Color Channels', size=30)
plt.show()

cv2.imwrite("images/me_aligned.png", im_aligned)

### Face Alignment

In [0]:
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import imutils
import dlib
import cv2
import matplotlib.pylab as plt

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor and the face aligner
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')
face_aligner = FaceAligner(predictor, desiredFaceWidth=256) 
# load the input image, resize it, and convert it to grayscale
image = cv2.imread('images/scientists.png')
image = imutils.resize(image, width=800)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
# show the original input image and detect faces in the grayscale image
plt.figure(figsize=(20,20))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)), plt.axis('off')
plt.title('Original Image: Famous Indian Scientists', size=20)
plt.show()
rects = detector(gray, 2)
print('Number of faces detected:', len(rects))
i = 1
# loop over the face detections
plt.figure(figsize=(10,20))
plt.subplots_adjust(0,0,1,1,0.05,0.12)
for rect in rects:
    # extract the ROI of the *original* face, then align the face
    # using facial landmarks
    (x, y, w, h) = rect_to_bb(rect)
    face_original = imutils.resize(image[y:y + h, x:x + w], width=256)
    face_aligned = face_aligner.align(image, gray, rect)

    # display the output images
    plt.subplot(9,4,i), plt.imshow(cv2.cvtColor(face_original, cv2.COLOR_BGR2RGB)), plt.title("Original", size=15), plt.axis('off')
    plt.subplot(9,4,i+1), plt.imshow(cv2.cvtColor(face_aligned, cv2.COLOR_BGR2RGB)), plt.title("Aligned", size=15), plt.axis('off')
    i += 2
plt.show()

In [0]:
import dlib
import cv2
from imutils import face_utils
from skimage.transform import AffineTransform, warp
import numpy as np
import matplotlib.pylab as plt

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor and the face aligner
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')
# load the input image, resize it, and convert it to grayscale
image = cv2.imread('images/monalisa.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 2)

faces = []
face_landmarks = []
for (i, rect) in enumerate(rects):
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    (left, top, w, h) = face_utils.rect_to_bb(rect)
    faces.append(image[top:top+h, left:left+w])
    landmark = []
    for (x, y) in shape:
        cv2.circle(image, (x, y), 1, (0, 255, 0), 2)
        landmark.append([x-left,y-top])
    face_landmarks.append(np.array(landmark))
        
plt.figure(figsize=(20,13))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Original image with Facial landmarks', size=20)
plt.show()
plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(cv2.cvtColor(faces[0], cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Right Face', size=20)
plt.subplot(122), plt.imshow(cv2.cvtColor(faces[1], cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Left Face', size=20)
plt.show()

transform = AffineTransform()
transform.estimate(face_landmarks[0], face_landmarks[1])
plt.figure(figsize=(10,10))
plt.gray()
plt.imshow(warp(cv2.cvtColor(faces[1], cv2.COLOR_BGR2RGB), transform, output_shape=faces[0].shape)), plt.axis('off'), plt.title('Warped right image on the left image', size=20)
plt.show()

### Face Morphing

In [0]:
from scipy.spatial import Delaunay
from scipy import interpolate
from skimage.io import imread
import scipy.misc
import cv2
import dlib
import numpy as np
from matplotlib import pyplot as plt

# Find 68 face landmarks using dlib
def get_face_landmarks(image, predictor_path = 'models/shape_predictor_68_face_landmarks.dat'):
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor(predictor_path)
  try:
    dets = detector(image, 1)
    points = np.zeros((68, 2))
    for k, d in enumerate(dets):
        # get the landmarks for the face in box d.
        shape = predictor(image, d)
        for i in range(68):
            points[i, 0] = shape.part(i).x
            points[i, 1] = shape.part(i).y
  except Exception as e:
    print('Failed finding face points: ', e)
    return []
  points = points.astype(np.int32)
  return points

def weighted_average_points(start_points, end_points, percent=0.5):
  # Weighted average of two sets of supplied points
  if percent <= 0:   return end_points
  elif percent >= 1: return start_points
  else: return np.asarray(start_points*percent + end_points*(1-percent), np.int32)

def weighted_average(img1, img2, percent=0.5):
  if percent <= 0: return img2
  elif percent >= 1: return img1
  else: return cv2.addWeighted(img1, percent, img2, 1-percent, 0)

# interpolates over every image channel
def bilinear_interpolate(img, coords):
  int_coords = coords.astype(np.int32)
  x0, y0 = int_coords
  dx, dy = coords - int_coords
  # 4 neighour pixels
  q11, q21, q12, q22 = img[y0, x0], img[y0, x0+1], img[y0+1, x0], img[y0+1, x0+1]
  btm = q21.T * dx + q11.T * (1 - dx)
  top = q22.T * dx + q12.T * (1 - dx)
  interpolated_pixels = top * dy + btm * (1 - dy)
  return interpolated_pixels.T

# generate x,y grid coordinates within the ROI of supplied points
def get_grid_coordinates(points):
  xmin, xmax = np.min(points[:, 0]), np.max(points[:, 0]) + 1
  ymin, ymax = np.min(points[:, 1]), np.max(points[:, 1]) + 1
  return np.asarray([(x, y) for y in range(ymin, ymax) for x in range(xmin, xmax)], np.uint32)

# warp each triangle from the src_image only within the ROI of the destination image (points in dst_points).
def process_warp(src_img, result_img, tri_affines, dst_points, delaunay):
  roi_coords = get_grid_coordinates(dst_points)
  # indices to vertices. -1 if pixel is not in any triangle
  roi_tri_indices = delaunay.find_simplex(roi_coords)
  for simplex_index in range(len(delaunay.simplices)):
    coords = roi_coords[roi_tri_indices == simplex_index]
    num_coords = len(coords)
    out_coords = np.dot(tri_affines[simplex_index], np.vstack((coords.T, np.ones(num_coords))))
    x, y = coords.T
    result_img[y, x] = bilinear_interpolate(src_img, out_coords)

# calculate the affine transformation matrix for each triangle vertex (x,y) from dest_points to src_points
def gen_triangular_affine_matrices(vertices, src_points, dest_points):
  ones = [1, 1, 1]
  for tri_indices in vertices:
    src_tri = np.vstack((src_points[tri_indices, :].T, ones))
    dst_tri = np.vstack((dest_points[tri_indices, :].T, ones))
    mat = np.dot(src_tri, np.linalg.inv(dst_tri))[:2, :]
    yield mat

def warp_image(src_img, src_points, dest_points, dest_shape):
  num_chans = 3
  src_img = src_img[:, :, :3]
  rows, cols = dest_shape[:2]
  result_img = np.zeros((rows, cols, num_chans), np.uint8)
  delaunay = Delaunay(dest_points)
  tri_affines = np.asarray(list(gen_triangular_affine_matrices(delaunay.simplices, src_points, dest_points)))
  process_warp(src_img, result_img, tri_affines, dest_points, delaunay)
  return result_img, delaunay

def read_lion_landmarks():    
    with open("models/lion_face_landmark.txt") as key_file:
        keypoints = [list(map(int, line.split())) for line in key_file]
        return(np.array(keypoints))

# load images
src_path = 'images/me.png'
dst_path = 'images/lion.png' 

src_img = imread(src_path)
dst_img = imread(dst_path)

size = dst_img.shape[:2]

src_img = cv2.resize(src_img[...,:3], size)

# define control points for warps
src_points = get_face_landmarks(src_img)
dst_points = read_lion_landmarks()

points = weighted_average_points(src_points, dst_points, percent=50)
src_face, src_delaunay = warp_image(src_img, src_points, points, size)
end_face, end_delaunay = warp_image(dst_img, dst_points, points, size)

print('here', len(src_points), len(dst_points))
  
fig = plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(src_img)
for i in range(len(src_points)):
    plt.plot(src_points[i,0], src_points[i,1], 'r.', markersize=20)
plt.title('Source image', size=20), plt.axis('off')
plt.subplot(122), plt.imshow(dst_img)
for i in range(len(dst_points)):
    plt.plot(dst_points[i,0], dst_points[i,1], 'g.', markersize=20)
plt.title('Destination image', size=20), plt.axis('off')
plt.suptitle('Facial Landmarks computed for the images', size=30)
fig.subplots_adjust(wspace=0.01, left=0.1, right=0.9)
plt.show()
fig = plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(src_img)
plt.triplot(src_points[:,0], src_points[:,1], src_delaunay.simplices.copy())
plt.plot(src_points[:,0], src_points[:,1], 'o', color='red'), plt.title('Source image', size=20), plt.axis('off')
plt.subplot(122), plt.imshow(dst_img)
plt.triplot(dst_points[:,0], dst_points[:,1], end_delaunay.simplices.copy())
plt.plot(dst_points[:,0], dst_points[:,1], 'o'), plt.title('Destination image', size=20), plt.axis('off')
plt.suptitle('Delaunay triangulation of the images', size=30)
fig.subplots_adjust(wspace=0.01, left=0.1, right=0.9)
plt.show()

fig = plt.figure(figsize=(18,20))
fig.subplots_adjust(top=0.925, bottom=0, left=0, right=1, wspace=0.01, hspace=0.08)
i = 1
for percent in np.linspace(1, 0, 16):
    points = weighted_average_points(src_points, dst_points, percent)
    src_face, src_delaunay = warp_image(src_img, src_points, points, size)
    end_face, end_delaunay = warp_image(dst_img, dst_points, points, size)
    average_face = weighted_average(src_face, end_face, percent)
    plt.subplot(4,4,i), plt.imshow(average_face), plt.title('alpha=' + str(round(percent,4)), size=20), plt.axis('off')
    i += 1
plt.suptitle('Face morphing', size=30)
plt.show()

### Robust Matching with RANSAC Algorithm using Harris Corner Brief Descriptors
[Praca badawcza](https://www.researchgate.net/publication/292995470_Image_Features_Detection_Description_and_Matching)

In [0]:
from skimage.feature import (corner_harris, corner_peaks, BRIEF, match_descriptors, plot_matches)
from skimage.transform import ProjectiveTransform, warp
from skimage.measure import ransac
from skimage.io import imread
from skimage.color import rgb2gray
import matplotlib.pylab as plt

np.random.seed(2)

img1 = rgb2gray(imread('images/victoria3.png'))
img2 = rgb2gray(imread('images/victoria4.png'))
keypoints1 = corner_peaks(corner_harris(img1), min_distance=1)
keypoints2 = corner_peaks(corner_harris(img2), min_distance=1)
extractor = BRIEF(patch_size=10)
extractor.extract(img1, keypoints1)
descriptors1 = extractor.descriptors
extractor.extract(img2, keypoints2)
descriptors2 = extractor.descriptors
matches = match_descriptors(descriptors1, descriptors2)

src_keypoints = keypoints1[matches[:,0]]
dst_keypoints = keypoints2[matches[:,1]]

homography = ProjectiveTransform()
homography.estimate(src_keypoints, dst_keypoints)


homography_robust, inliers = ransac((src_keypoints, dst_keypoints), ProjectiveTransform, min_samples=4,
                               residual_threshold=2, max_trials=500)
outliers = inliers == False
print(len(matches))

robust_matches = match_descriptors(descriptors1[matches[:,0]][inliers], descriptors2[matches[:,1]][inliers])
print(len(robust_matches))

fig, ax = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(20,15))
plt.gray()
plt.subplots_adjust(0,0,1,1,0.05,0.05)
plot_matches(ax[0,0], img1, img2, keypoints1, keypoints2, matches), ax[0,0].set_title('Matching without RANSAC', size=20)
ax[0,1].imshow(warp(img2, homography, output_shape=img2.shape)), ax[0,1].set_title('Homography without RANSAC', size=20)
plot_matches(ax[1,0], img1, img2, keypoints1, keypoints2, robust_matches), ax[1,0].set_title('Robust Matching with RANSAC', size=20)
ax[1,1].imshow(warp(img2, homography_robust, output_shape=img2.shape)), ax[1,1].set_title('Robust Homography with RANSAC', size=20)
for a in np.ravel(ax):
    a.axis('off')
plt.show()

In [0]:

!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

### Image Mosaicing (Cylindrical Panorama)

In [0]:
import cv2
# for this problem let's work with opencv 3.4.2.16
print(cv2.__version__)
# 3.4.2
# pip install opencv-contrib-python==3.4.2.16
# pip install opencv-python==3.4.2.16
import numpy as np
from matplotlib import pyplot as plt
import math
import glob

def compute_homography(image1, image2, bff_match=False):

    sift = cv2.xfeatures2d.SIFT_create(edgeThreshold=10, sigma=1.5, contrastThreshold=0.08)
    
    kp1, des1 = sift.detectAndCompute(image1, None)
    kp2, des2 = sift.detectAndCompute(image2, None)

    # Brute force matching
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, trainDescriptors=des2, k=2)

    # Lowes Ratio
    good_matches = []
    for m, n in matches:
        if m.distance < .75 * n.distance:
            good_matches.append(m)

    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])\
        .reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])\
        .reshape(-1, 1, 2)

    if len(src_pts) > 4:
        H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5)
    else:
        H = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    return H


def warp_image(image, H):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    h, w, _ = image.shape

    # Find min and max x, y of new image
    p = np.array([[0, w, w, 0], [0, 0, h, h], [1, 1, 1, 1]])
    p_prime = np.dot(H, p)

    yrow = p_prime[1] / p_prime[2]
    xrow = p_prime[0] / p_prime[2]
    ymin = min(yrow)
    xmin = min(xrow)
    ymax = max(yrow)
    xmax = max(xrow)

    # Create a new matrix that removes offset and multiply by homography
    new_mat = np.array([[1, 0, -1 * xmin], [0, 1, -1 * ymin], [0, 0, 1]])
    H = np.dot(new_mat, H)

    # height and width of new image frame
    height = int(round(ymax - ymin))
    width = int(round(xmax - xmin))
    size = (width, height)
    # Do the warp
    warped = cv2.warpPerspective(src=image, M=H, dsize=size)

    return warped, (int(xmin), int(ymin))

def cylindrical_warp_image(img, H):
    
    h, w = img.shape[:2]
    # pixel coordinates
    y_i, x_i = np.indices((h, w))
    X = np.stack([x_i,y_i,np.ones_like(x_i)],axis=-1).reshape(h*w, 3) # to homog
    Hinv = np.linalg.inv(H) 
    X = Hinv.dot(X.T).T # normalized coords
    # calculate cylindrical coords (sin\theta, h, cos\theta)
    A = np.stack([np.sin(X[:,0]),X[:,1],np.cos(X[:,0])],axis=-1).reshape(w*h, 3)
    B = H.dot(A.T).T # project back to image-pixels plane
    # back from homog coords
    B = B[:,:-1] / B[:,[-1]]
    # make sure warp coords only within image bounds
    B[(B[:,0] < 0) | (B[:,0] >= w) | (B[:,1] < 0) | (B[:,1] >= h)] = -1
    B = B.reshape(h,w,-1)
    
    img_rgba = cv2.cvtColor(img,cv2.COLOR_BGR2BGRA) # for transparent borders...
    # warp the image according to cylindrical coords
    return cv2.remap(img_rgba, B[:,:,0].astype(np.float32), B[:,:,1].astype(np.float32), cv2.INTER_AREA, borderMode=cv2.BORDER_TRANSPARENT)

def create_mosaic(images, origins):
    # find central image
    for i in range(0, len(origins)):
        if origins[i] == (0, 0):
            central_index = i
            break

    central_image = images[central_index]
    central_origin = origins[central_index]
    
    # zip origins and images together
    zipped = list(zip(origins, images))
    
    # sort by distance from origin (highest to lowest)
    func = lambda x: math.sqrt(x[0][0] ** 2 + x[0][1] ** 2)
    dist_sorted = sorted(zipped, key=func, reverse=True)
    # sort by x value
    x_sorted = sorted(zipped, key=lambda x: x[0][0])
    # sort by y value
    y_sorted = sorted(zipped, key=lambda x: x[0][1])

    # determine the coordinates in the new frame of the central image
    if x_sorted[0][0][0] > 0:
        cent_x = 0  # leftmost image is central image
    else:
        cent_x = abs(x_sorted[0][0][0])

    if y_sorted[0][0][1] > 0:
        cent_y = 0  # topmost image is central image
    else:
        cent_y = abs(y_sorted[0][0][1])

    # make a new list of the starting points in new frame of each image
    spots = []
    for origin in origins:
        spots.append((origin[0]+cent_x, origin[1] + cent_y))

    zipped = zip(spots, images)

    # get height and width of new frame
    total_height = 0
    total_width = 0

    for spot, image in zipped:
        total_width = max(total_width, spot[0]+image.shape[1])
        total_height = max(total_height, spot[1]+image.shape[0])

    # print "height ", total_height
    # print "width ", total_width

    # new frame of panorama
    stitch = np.zeros((total_height, total_width, 4), np.uint8)

    # stitch images into frame by order of distance
    for image in dist_sorted:
        
        offset_y = image[0][1] + cent_y
        offset_x = image[0][0] + cent_x
        end_y = offset_y + image[1].shape[0]
        end_x = offset_x + image[1].shape[1]
        
        ####
        stitch_cur = stitch[offset_y:end_y, offset_x:end_x, :4]
        stitch_cur[image[1]>0] = image[1][image[1]>0]
        ####
                
        #stitch[offset_y:end_y, offset_x:end_x, :4] = image[1]

    return stitch

def create_panorama(images, center):
    
    h,w,_ = images[0].shape
    f = 1000 # 800
    H = np.array([[f, 0, w/2], [0, f, h/2], [0, 0, 1]])
    for i in range(len(images)):
        images[i] = cylindrical_warp_image(images[i], H)
    
    panorama = None
    for i in range(center):
        print('Stitching images {}, {}'.format(i+1, i+2))
        image_warped, image_origin = warp_image(images[i], compute_homography(images[i + 1], images[i]))
        panorama = create_mosaic([image_warped, images[i+1]], [image_origin, (0,0)])
        images[i + 1] = panorama

    #print('Done left part')

    for i in range(center, len(images)-1):
        print('Stitching images {}, {}'.format(i+1, i+2))
        image_warped, image_origin = warp_image(images[i+1], compute_homography(images[i], images[i + 1]))
        panorama = create_mosaic([images[i], image_warped], [(0,0), image_origin])
        images[i + 1] = panorama

    #print('Done right part')
    return panorama

images = [ cv2.cvtColor(cv2.imread(img), cv2.COLOR_RGB2RGBA) for img in glob.glob('images/victoria*.png')]

plt.figure(figsize=(20,4))
plt.subplots_adjust(top = 0.8, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0.05)
plt.margins(0,0)
for i in range(len(images)):
    plt.subplot(1,len(images),i+1), plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Image {}'.format(i+1), size=15)
plt.suptitle('Images to Stitch', size=20)
plt.show()

center = len(images) // 2
#print(len(images), center)

panorama = create_panorama(images, center)

plt.figure(figsize=(20,8))
plt.subplots_adjust(top = 0.9, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
plt.margins(0,0)
plt.imshow(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Final Panorama Image', size=15)
plt.show()

### Panorama with opencv-python

In [0]:
import numpy as np
import cv2
import glob
import matplotlib.pylab as plt

print(cv2.__version__)
# 3.4.2

# grab the paths to the input images and initialize our images list
print("Loading images...")
images = [ cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB) for img in glob.glob('images/victoria*.png')]
print('Number of images to stitch: {}'.format(len(images)))
fig = plt.figure(figsize=(20, 5))
for i in range(len(images)):
    plt.subplot(1,len(images),i+1)
    plt.imshow(images[i])
    plt.axis('off')
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.95, hspace=0.05, wspace=0.05) 
plt.suptitle('Images to stich', size=25)
plt.show()

# initialize OpenCV's image sticher object and then perform the image
# stitching
print("Stitching images...")
stitcher = cv2.createStitcher()
(status, stitched) = stitcher.stitch(images)
print(status)

plt.figure(figsize=(20,10))
plt.imshow(stitched), plt.axis('off'), plt.title('Final Panorama Image', size=20)
plt.show()

### Finding similarity between an image and a set of images

In [0]:
import cv2
print(cv2.__version__)
# 3.4.2
import numpy as np
import glob
import matplotlib.pylab as plt
from collections import defaultdict

query = cv2.imread("images/query.png", cv2.CV_8U)

matched_images = defaultdict(list)
for image_file in glob.glob('images/search/*.png'):
    
    search_image = cv2.imread(image_file, cv2.CV_8U)
    sift = cv2.xfeatures2d.SIFT_create()
    kp_1, desc_1 = sift.detectAndCompute(query, None)
    kp_2, desc_2 = sift.detectAndCompute(search_image, None)
    index_params = dict(algorithm=0, trees=5)
    search_params = dict()
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desc_1, desc_2, k=2)
    good_points = []
    ratio = 0.6
    for m, n in matches:
        if m.distance < ratio*n.distance:
            good_points.append(m)
    num_good_points = len(good_points)
    print('Image file = {}, Number of good matches = {}'.format(image_file, num_good_points))
    if (num_good_points > 300) or (num_good_points < 10):
        result = cv2.drawMatches(query, kp_1, search_image, kp_2, good_points, None)
        plt.figure(figsize=(20,10))
        plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB)), plt.axis('off')
        plt.title(('Good match' if num_good_points > 300 else 'Poor match') + ' with {} matches'.format(num_good_points), size=20)
        plt.show()

    matched_images[len(good_points)].append(search_image)

plt.figure(figsize=(15,10))
plt.gray()
plt.imshow(query), plt.axis('off')
plt.title('Original (Query) Image', size=20)
plt.show()

i = 1
plt.figure(figsize=(20,35))
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.925, wspace=0.02, hspace=0.1)
for num_matches in sorted(matched_images, reverse=True):
    for image in matched_images[num_matches]:
        plt.subplot(10, 4, i)
        plt.imshow(image)
        plt.axis('off')
        plt.title('Image with {} good matches'.format(num_matches), size=15)
        i += 1
plt.suptitle('Images matched with the Query Image ranked by the number of good matches', size=20)
plt.show()

In [0]:
query.shape

In [0]:
#https://www.kaggle.com/duttadebadri/image-classification/downloads/image-classification.zip/2